### 의사결정나무와 랜덤 포레스트(예측/분류모델)   

의사결정나무란 데이터가 트리 구조로 분기되며 최적의 예측 조건을 찾는 알고리즘이다.    
분류와 회귀 모두 사용 가능하다    
기본 원리: 데이터를 불순도가 낮아지도록 반복적으로 나누면서 트리를 생성   
범주의 동질성이 최대한 높도록 분류 기준을 설정    

불순도 측정 방법    
지니 계수: 그룹 내 특정 클래스 비율의 제곱을 이용해 계산    
  - 범위: 0(완전 순수) ~ 0.5(불순도 최대)    

엔트로피: 불확실성을 측정하여 분류 성능을 평가    
  - 범위: 0(완전 순수) ~ log2(c)(불순도 최대)   

정보 획득량: 분류 전후의 불손도 감소량을 측정   
  - 정보 획득량이 높을수록 분할이 효과적    
  - 의사결정나무는 정보 획득량이 가장 높은 변수를 선택하여 분할 진행    

의사결정나무의 한계   
과적합 문제   
트리의 깊이가 너무 깊어지면 데이터에 과적합되어 새로운 데이터에 대한 예측력이 저하된다. 
해결 방법: 가지치기, 노드 데이터 수 제한, 트리 깊이 제한      

불안정한 결과   
새로운 데이터가 추가되면 트리의 분기가 달라질 수 있어 일관된 모델 유지가 어렵다.    
해결 방법: 랜덤 포레스트 사용   

랜덤 포레스트란 의사결정나무의 단점을 보완한 앙상블 학습 기법이다.    
여러 개의 의사결정나무를 학습시켜 예측을 종합하는 방식으로 단일 의사결정나무 모델보다 일반화 성능이 우수하고, 과적합을 방지해준다.    

기본원리    
여러 개의 의사결정나무를 만들어 다수결(분류) 또는 평균(회귀)으로 최종 에측을 수행   
데이터와 독립변수를 무작위로 샘플링하여 각각의 트리에 학습시키는 방식   
트리 간의 상관성을 줄여 과적합을 방지하고 모델의 안정성을 높인다.   

학습 및 예측 과정   
Step 1: 학습 데이터에서 부트스트랩 방식으로 랜덤 샘플링   
Step 2: 각 트리에 사용할 독립변수를 무작위로 n개 선택(중복 없음)    
Step 3: 선택한 데이터와 변수로 의사결정나무 학습      
Step 4: Step1~3을 k번 반복하여 여러 개의 트리 생성    
Step 5: 생성한 k개의 트리를 통해 예측값 도출    
분류인 경우 다수결 방식으로 최종 결과 예측    
회귀인 경우 각 트리의 예측값 평균으로 최종 결과 예측       

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uciml/glass")

print("Path to dataset files:", path)

100%|██████████| 3.42k/3.42k [00:00<00:00, 3.50MB/s]

Extracting files...
Path to dataset files: C:\Users\KimDongyoung\.cache\kagglehub\datasets\uciml\glass\versions\1


In [3]:
# 필요한 패키지 설치
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
# import pydotplus
from sklearn.tree import export_graphviz
from IPython.core.display import Image
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
# 데이터 불러오기
# https://www.kaggle.com/datasets/uciml/glass
df = pd.read_csv("C:/Users/KimDongyoung/.cache/kagglehub/datasets/uciml/glass/versions/1/glass.csv")

# 데이터 샘플 확인
df.head()


,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [5]:
# 컬럼 정보 확인

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RI      214 non-null    float64
 1   Na      214 non-null    float64
 2   Mg      214 non-null    float64
 3   Al      214 non-null    float64
 4   Si      214 non-null    float64
 5   K       214 non-null    float64
 6   Ca      214 non-null    float64
 7   Ba      214 non-null    float64
 8   Fe      214 non-null    float64
 9   Type    214 non-null    int64  
dtypes: float64(9), int64(1)
memory usage: 16.8 KB


In [8]:
# 종속변수 문자형 변환

df["Type_str"]=df["Type"].apply(str)
df.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type,Type_str
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1,1


In [9]:
# 의사결정나무 모델 용 데이터셋 가공

# 독립변수, 종속변수 분리
df_x = df[['RI','Na','Mg','Al','Si','K','Ca','Ba','Fe']]
df_y = df[['Type_str']]

# 학습셋과 테스트셋 분리하여 생성(6:4)
x_train, x_test, y_train, y_test = train_test_split(
    df_x,df_y,test_size=0.4,random_state=10)

# 학습셋과 검증셋이 잘 나뉘었는지 확인
print('train data 개수: ', len(x_train))
print('test data 개수: ', len(x_test))

train data 개수:  128
test data 개수:  86


In [10]:
# 의사결정나무 모델 적용

# 모델 생성 및 학습
dt_model = DecisionTreeClassifier(criterion = "entropy", max_depth=5)
dt_model.fit(x_train,y_train)

# 테스트셋 예측
y_pred = dt_model.predict(x_test)

# 정확도 계산
accuracy_score(y_test, y_pred)

0.6046511627906976

In [11]:
# 독립변수 중요도 확인

dt_model.feature_importances_

pd.DataFrame({'feature' : x_train.columns,

              'importance' : dt_model.feature_importances_})


,feature,importance
0,RI,0.071871
1,Na,0.082840
2,Mg,0.320142
3,Al,0.237760
4,Si,0.155650
5,K,0.017020
6,Ca,0.114717
7,Ba,0.000000
8,Fe,0.000000


In [13]:
# 랜덤포레스트 모델 생성

# 기본 모델
# rnf_model = RandomForestClassifier(n_estimators=100, max_depth=5,random_state=0)

rnf_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0,
                                   bootstrap=True, class_weight=None, criterion='gini',
                                   max_features='sqrt', max_leaf_nodes=None,
                                   min_impurity_decrease=0.0, min_samples_leaf=1, 
                                   min_samples_split=2, min_weight_fraction_leaf=0.0, 
                                   n_jobs=None, oob_score=False,  verbose=0,
                                   warm_start=False)

rnf_model.fit(x_train,y_train)

y_pred = rnf_model.predict(x_test)
print(accuracy_score(y_test,y_pred))


0.7093023255813954


c:\Users\KimDongyoung\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [14]:
# 랜덤포레스트 모델 성능 리포트 출력

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.69      0.73      0.71        30
           2       0.62      0.77      0.69        26
           3       0.67      0.29      0.40         7
           5       1.00      0.75      0.86         8
           6       0.50      0.25      0.33         4
           7       0.91      0.91      0.91        11

    accuracy                           0.71        86
   macro avg       0.73      0.62      0.65        86
weighted avg       0.72      0.71      0.70        86



In [15]:
# 랜덤포레스트 변수 중요도 출력

feature_imp = pd.Series(rnf_model.feature_importances_, 
                        index=x_train.columns).sort_values(ascending=False)
feature_imp[:20]


Al    0.159236
Ca    0.153955
Mg    0.148816
RI    0.140340
Na    0.107937
Ba    0.090783
K     0.079957
Si    0.072258
Fe    0.046717
dtype: float64